# Import Packages

In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, PorterStemmer
import math
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, mean_squared_error
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
import gensim
import string
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import CuDNNLSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
import warnings
from keras import backend as K
warnings.filterwarnings("ignore")
#K.tensorflow_backend._get_available_gpus()
from wordcloud import WordCloud, STOPWORDS

Using TensorFlow backend.


In [2]:
import keras.backend.tensorflow_backend as tfback

print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

tf.__version__ is 2.1.0
tf.keras.__version__ is: 2.2.4-tf


# Import training and testing dataset 

In [3]:
train_df = pd.read_csv(r'D:\Springboard\Github\Data-NLP-Toxic Classification\train.csv', index_col='id',engine='python')
train_df.head()

,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
id,,,,,,,,,,,,,,,,,,,,,
59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [4]:
test_df = pd.read_csv(r'D:\Springboard\Github\Data-NLP-Toxic Classification\test.csv',index_col='id',engine='python')
test_df.head()

,comment_text
id,
7097320,[ Integrity means that you pay your debts.]\n\...
7097321,This is malfeasance by the Administrator and t...
7097322,@Rmiller101 - Spoken like a true elitist. But ...
7097323,"Paul: Thank you for your kind words. I do, in..."
7097324,Sorry you missed high school. Eisenhower sent ...


# Preprocessing the Training and Testing Dataset 

In [39]:
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))
def preprocess(text_string):
    text_string = text_string.lower() # Convert everything to lower case.
    text_string = re.sub('[^A-Za-z0-9]+', ' ', text_string) # Remove special characters and punctuations
    
    x = text_string.split()
    new_text = []
    
    for word in x:
        if word not in stop_words:
            new_text.append(stemmer.stem(word))
            
    text_string = ' '.join(new_text)
    return text_string

In [42]:
%%time 
train_df['preprocessed_text'] = train_df['comment_text'].apply(preprocess)

Wall time: 12min 29s


In [50]:
train_df['preprocessed_text'].head()

id
59848    cool like would want mother read realli great ...
59849    thank would make life lot less anxieti induc k...
59852              urgent design problem kudo take impress
59855                        someth abl instal site releas
59856                                 haha guy bunch loser
Name: preprocessed_text, dtype: object

In [45]:
%%time 
test_df['preprocessed_text'] = test_df['comment_text'].apply(preprocess)

Wall time: 35.8 s


In [41]:
test_df['preprocessed_text']

id
7097320              integr mean pay debt appli presid trump
7097321                   malfeas administr board wast money
7097322    rmiller101 spoken like true elitist look bud a...
7097323    paul thank kind word inde strong belief hide b...
7097324    sorri miss high school eisenhow sent troop vie...
                                 ...                        
7194635              lose job promot mis inform harm student
7194636    thin project meant lower fire danger improv wi...
7194637                hope millenni happi put airhead charg
7194638    think kellyann conway k trump whisper play lar...
7194639    still figur pizza ak cost pizza washington ida...
Name: preprocessed_text, Length: 97320, dtype: object

# Feature Engineering

Define the predicted variable and targer varible for the modeling  

In [51]:
X = train_df[['preprocessed_text']]
y = train_df[['target']]

In [56]:
X_train,X_cv, y_train, y_cv = train_test_split(X,y)
print(X_train.shape)
print(X_cv.shape)
print(y_train.shape)
print(y_cv.shape)

(1353655, 1)
(451219, 1)
(1353655, 1)
(451219, 1)


In [57]:
X_train.head()

,preprocessed_text
id,
475441,see word black articl big bad gangster thug gu...
6296990,caldwel help ige run governor term run chin pl...
5274068,al green may correct trump action warrant impe...
5340438,lebron respons classi dignifi could gone rant ...
6293850,california municip cooper ice hand crimin insi...


In [58]:
X_cv.head()

,preprocessed_text
id,
5876537,exact give charli mnof favorit troll cue even ...
325502,jpw uof0 zero administr act like charg requir ...
5449922,must difficult thing democrat america today ex...
5059513,saw can heat earli 70s may 72 saw blue oyster ...
567832,tell truth day micro aggress make hillari supp...


In [59]:
X_test = test_df[['preprocessed_text']]
X_test.head()

,preprocessed_text
id,
7097320,integr mean pay debt appli presid trump
7097321,malfeas administr board wast money
7097322,rmiller101 spoken like true elitist look bud a...
7097323,paul thank kind word inde strong belief hide b...
7097324,sorri miss high school eisenhow sent troop vie...


# Save the dataset  separately for modeling 

In [60]:
X_train.to_pickle('X_train.pkl')
X_cv.to_pickle('X_cv.pkl')
X_test.to_pickle('X_test.pkl')
y_train.to_pickle('y_train.pkl')
y_cv.to_pickle('y_cv.pkl')